In [1]:
import pandas as pd
import numpy as np

import neologdn
import MeCab

import re

### neologdnで正規化、Mecab + neologd 辞書による形態素解析を行い、形態素解析結果をデータフレームにする関数

In [2]:
def get_wakati_text_df(text):
    text_normalization = neologdn.normalize(text)
    neologd_tagger = MeCab.Tagger('-Ochasen -d C:\mecab-ipadic-neologd')
    
    # neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して表示される
    # 原形、品詞などの間には「\t」が、分かち書きされた単語と単語の区切りには「\n」が表示される
    # まずはparse()で分かち書きした単語群は1つの文字列型になっているので「\n」で区切り、リスト型にする
    wakatid_text = neologd_tagger.parse(text_normalization).split('\n')
    
    # 次に邪魔な文字「\t」を省きたい。よって、split('\t')を使用したいがwakatid_textは文字列型ではなくリスト型のためsplit関数を使用できない
    # そこで、リスト型のwakatid_textをまずSeries型に変更して「str（str アクセサ）」を呼び出せるようにした後にstrを使用してリストの内部の文字列に対してsplit関数を実行する
    ##（「str」がないとエラー：'Series' object has no attribute 'split'が発生する）
    ## pandas では内部（今回はリストの内部）のデータ型が文字列 ( str もしくは unicode ) 型のとき、 str アクセサを使ってデータの各要素に対して文字列メソッドを適用することができる
    ##今回は str アクセサを通じて、リスト内の各要素に split メソッドを適用
    ## 参考URL：http://sinhrks.hatenablog.com/entry/2014/12/06/233032
    # pd.Series(wakatid_text).str.split('\t')までで各要素をリスト型にしたものの、全体はSeries型のままである
    # そこで最後にtolist関数でリスト型にする
    # 結果、['た', 'タ', 'た', '助動詞', '特殊・タ', '基本形'],['時間', 'ジカン', '時間', '名詞-副詞可能', '', ''],といった単語毎にリスト化されている状態、リストの中にリストが作成されている
    wakatid_results = pd.Series(wakatid_text).str.split('\t').tolist()
    
    # リスト型のwakatid_resultsをデータフレーム型にする
    # pd.DataFrame.from_records 参考URL：http://pbpython.com/pandas-list-dict.html
    df = pd.DataFrame.from_records(wakatid_results)
    # カラムがまだないのでカラムを作成
    columns = ['surface', 'yomi', 'original', 'type', 'katsuyoukei', 'katsuyougata']
    # 作成データフレームのカラムを上記のカラムにする
    df.columns = columns
    # 'EOS’は最後から2つ目の要素（最後の要素は「['']」）
    # これら2つ以外のsurfaceを格納したデータフレームを返り値にする
    return df.query("surface != 'EOS'").query("surface != ''")

### 関数の実行

In [3]:
get_wakati_text_df('空いた時間でリサーチデザイン、交絡因子、統計的コントロールなど計量社会科学を勉強した。兎にも角にも「同じ世界で比較してやる」という方針で分析設計をする感じなのかな。')

,surface,yomi,original,type,katsuyoukei,katsuyougata
0,空い,アイ,空く,動詞-自立,五段・カ行イ音便,連用タ接続
1,た,タ,た,助動詞,特殊・タ,基本形
2,時間,ジカン,時間,名詞-副詞可能,,
3,で,デ,で,助詞-格助詞-一般,,
4,リサーチ,リサーチ,リサーチ,名詞-サ変接続,,
5,デザイン,デザイン,デザイン,名詞-サ変接続,,
6,、,、,、,記号-読点,,
7,交,ヨシミ,交,名詞-固有名詞-人名-名,,
8,絡,絡,絡,名詞-固有名詞-組織,,
9,因子,インシ,因子,名詞-一般,,


##### 参考URL：https://datumstudio.jp/blog/python%e3%81%ab%e3%82%88%e3%82%8b%e6%97%a5%e6%9c%ac%e8%aa%9e%e5%89%8d%e5%87%a6%e7%90%86%e5%82%99%e5%bf%98%e9%8c%b2